In [11]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [12]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [13]:
# Path of the CSV file
csvfile1 = "Resources/hawaii_measurements_clean.csv"
csvfile2 = "Resources/hawaii_stations_clean.csv"

In [14]:
# Read CSV file into a pandas DataFrame
df1 = pd.read_csv(csvfile1, dtype=object)
df2 = pd.read_csv(csvfile2, dtype=object)

In [15]:
# Preview DataFrame
print (df1.head())
print (df2.head())

       station        date  prcp tobs
0  USC00519397  2010-01-01  0.08   65
1  USC00519397  2010-01-02     0   63
2  USC00519397  2010-01-03     0   74
3  USC00519397  2010-01-04     0   76
4  USC00519397  2010-01-07  0.06   70
       station                                    name latitude  longitude  \
0  USC00519397                    WAIKIKI 717.2, HI US  21.2716  -157.8168   
1  USC00513117                    KANEOHE 838.1, HI US  21.4234  -157.8015   
2  USC00514830  KUALOA RANCH HEADQUARTERS 886.9, HI US  21.5213  -157.8374   
3  USC00517948                       PEARL CITY, HI US  21.3934  -157.9751   
4  USC00518838              UPPER WAHIAWA 874.3, HI US  21.4992  -158.0111   

  elevation  
0         3  
1      14.6  
2         7  
3      11.9  
4     306.6  


In [16]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [17]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [18]:
# Create a connection to the engine called `connection`
conn = engine.connect()

In [19]:
#Use declarative_base and create ORM classes for each table
Base = declarative_base()

class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [20]:
# Use `create_all` to create the table in the database
Base.metadata.create_all(engine)

In [22]:
# Use Orient='records' to create a list of measurement data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
### BEGIN SOLUTION
data1 = df1.to_dict(orient='records')
data1[0]
### END SOLUTION

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [23]:
# Use Orient='records' to create a list of station data to write
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
### BEGIN SOLUTION
data2 = df2.to_dict(orient='records')
data2[0]
### END SOLUTION

{'elevation': '3',
 'latitude': '21.2716',
 'longitude': '-157.8168',
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [25]:
# Use MetaData from SQLAlchemy to reflect the tables
### BEGIN SOLUTION
metadata = MetaData(bind=engine)
metadata.reflect()
### END SOLUTION

In [26]:
# Save the reference to the `measurement` table as a variable called `measurement_table`
### BEGIN SOLUTION
measurement_table = sqlalchemy.Table('measurement', metadata, autoload=True)
### END SOLUTION

In [27]:
# Save the reference to the `measurement` table as a variable called `station_table`
### BEGIN SOLUTION
station_table = sqlalchemy.Table('station', metadata, autoload=True)
### END SOLUTION

In [28]:
# Use `measurement_table.insert()` to insert the measurement data into the measurement table
### BEGIN SOLUTION
conn.execute(measurement_table.insert(), data1)
### END SOLUTION

In [29]:
# Use `station_table.insert()` to insert the data into the station table
### BEGIN SOLUTION
conn.execute(station_table.insert(), data2)
### END SOLUTION

In [30]:
# Test that the insert works by fetching the first 5 rows for measurement. 
conn.execute("select * from measurement limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [31]:
# Test that the insert works by fetching the first 5 rows for station. 
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]